In [182]:
from scipy.io import arff
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import json

## Synthetic ts

In [183]:
def generate_ARHT(t_0, T, num_harmonics = 0, coeffs = None, periods = None, max_period = 100, min_period = 0,
                  trend = False, power = 0, const = 0, var = 1):
    signal = np.zeros(T - t_0 + 1)
    t = np.arange(t_0,T+1)
    if num_harmonics: #todo: Check consistant number of harmonics length of coeffs and periods
        if coeffs is None:
            coeffs = 2 * np.random.random(harmonics_components) + 0.3
        if periods is None:
            sine_periods = ((max_period-min_period)* np.random.random(num_harmonics) + min_period)
        for i in range(num_harmonics): 
            signal += coeffs[i] * np.sin((t/(T - t_0 + 1))*periods[i]*2.0*np.pi)
    if trend: #add more trend types
        signal += t**power
    if const: 
        signal += const
    noise = np.random.normal(0.0, np.sqrt(var), T - t_0 + 1)
    noisy_ts = signal + noise
    return noisy_ts, signal



In [188]:
def generate_synthetic(cp_type, cp_number, T, nh, trend, noise_var, power = 0): 
    ts = np.array([])
    tm = np.array([])
    if cp_type == 'struct': 
        const = np.zeros(cp_number)
        coeffs = np.tile(np.random.random(nh)*7, cp_number).reshape(cp_number,nh)
        periods = np.zeros((cp_number, nh))
        for i in range(cp_number): 
            if i%2==0: 
                periods[i, :] = 5 + np.random.random(nh) * 30
            else: 
                periods[i, :] = 5 + np.random.random(nh) * 70
    if cp_type == 'mean': 
        const = np.random.random(cp_number) * 10
        coeffs = np.tile(np.random.random(nh)*7, cp_number).reshape(cp_number,nh)
        periods = np.tile(5+np.random.random((nh))*100, cp_number).reshape(cp_number,nh)
    if cp_type == 'energy': 
        const = np.zeros(cp_number)
        periods = np.tile(5+np.random.random((nh))*100, cp_number).reshape(cp_number,nh)
        coeffs = np.zeros((cp_number, nh))
        for i in range(cp_number): 
            if i%2==0: 
                coeffs[i, :] = np.random.random() * 7 * np.ones(nh)
            else: 
                coeffs[i, :] = np.random.random() * 5 * np.ones(nh)
    param = {}
    for i in range(0, cp_number):
        key = 'ts_{}'.format(i)
        param[key] = {'periods': periods[i], 'coeffs': coeffs[i], 'conts': const[i], 'noice': noise_var, 'trend_power': power}
        ts_, tm_ = generate_ARHT(i*T + 1, (i+1)*T, num_harmonics = nh, coeffs = coeffs[i], periods = periods[i], 
                                 max_period = 100, min_period = 0, trend = trend, power = power, const = const[i], var = noise_var)
        ts = np.append(ts, ts_)
        tm = np.append(tm, tm_)
    start = np.arange(0, cp_number+1) * T
    end = np.arange(1, cp_number+2) * T - 1
    new_labels = pd.DataFrame(start)
    new_labels[1] = end
    return ts, tm, new_labels, param

        
        
        
        
        

In [189]:
def json_converter(obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, datetime.datetime):
            return obj.__str__()

In [195]:
cp_type = 'energy'
cp_number = 5 
T = 1000
nh = 3 
trend = False
noise_var = 0.01
samples = 20
data_path = os.path.join(os.path.dirname(os.getcwd()), 'data', 'synth')
for sample in range(samples): 
    ts, tm, labels, param = generate_synthetic(cp_type, cp_number, T, nh, trend, noise_var, power = 0)
    df = pd.DataFrame(ts)
    df.reset_index(inplace=True)
    df.to_csv(path_or_buf=os.path.join(data_path, '{}{}_ts.csv'.format(cp_type, sample)), index=False, header=False)
    df = pd.DataFrame(tm)
    df.reset_index(inplace=True)
    df.to_csv(path_or_buf=os.path.join(data_path, '{}{}_tm.csv'.format(cp_type, sample)), index=False, header=False)
    labels.to_csv(path_or_buf=os.path.join(data_path, '{}{}_labels.csv'.format(cp_type, sample)), index=False, header=False)
    with open(os.path.join(data_path, '{}{}_param.csv'.format(cp_type, sample)), 'w') as file:
        json.dump(param, file, default=json_converter)
    